In [ ]:
# import necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataaset with features
df = pd.read_csv("../dataset/processed_data/dataset_with_feature_new.csv")

In [ ]:
df.head()

In [ ]:
# Convert categorical features to numerical values
# df = pd.get_dummies(df, columns=['prefix', 'checksum'])

In [ ]:
le = LabelEncoder()

In [ ]:
# df['prefix'] = le.fit_transform(df['prefix'])
# df['checksum'] = le.fit_transform(df['checksum'])

In [ ]:
df = pd.get_dummies(df, columns=['prefix', 'checksum'])

In [ ]:
df.head()

In [ ]:
df.to_csv("trydata.csv", index=False)

In [ ]:
df.columns

In [ ]:
# Label encoding for target
df['type'] = le.fit_transform(df['type'])

In [ ]:
# Separate features and target
X = df.drop(columns=['address', 'type'])
y = df['type']

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [ ]:
# Save class information for inference
np.save('../models/classes.npy', le.classes_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
# Define models to train
models = {
    "RandomForest": RandomForestClassifier(n_estimators=150, random_state=62),
    # "LogisticRegression": LogisticRegression(max_iter=1000, random_state=58),
    "SVC": SVC(random_state=42),
    # "DecisionTree": DecisionTreeClassifier(random_state=23),
    "NaiveBayes": GaussianNB()
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"Results for {model_name}:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("\n")

    # Save the model
    joblib.dump(model, f"../models/{model_name}.pkl")
    print(f"{model_name} model saved as {model_name}.pkl\n")

    # Save Random Forest model to a variable for later use
    if model_name == "RandomForest":
        rf_model = model

In [ ]:
# Define the neural network architecture
nn_model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Output layer with 3 neurons for 3 classes
])

# Compile the model
nn_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = nn_model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
# Evaluate the model on the test set
loss, accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
print(f"Neural Network Test Accuracy: {accuracy:.4f}")

# Predict on the test set
y_pred = nn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print classification report
print("Classification Report for Neural Network:")
print(classification_report(y_test, y_pred_classes))

# Save the trained neural network model
nn_model.save("../models/neural_network_model.keras")
print("Neural Network model saved as neural_network_model.keras")

In [ ]:
# check feature importance
# Get feature importances
feature_importances = rf_model.feature_importances_
feature_names = df.drop(columns=['address', 'type']).columns

# Create a DataFrame for feature importances
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance with respect to Target Variable')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()